### <center>Importações e carregamento dos dados

In [187]:
from scipy.stats import ttest_ind
import pandas as pd
import pingouin as pg
import plotly.express as px

# Carregar os dados do Excel
df_base = pd.read_excel("dados_ab_teste.xlsx")


### <center>Transformação em segundo e armazenamento em variaveis

In [188]:
# Converter a coluna "Tempo (min:seg)" para minutos decimais diretamente
df_base["Tempo (seg)"] = df_base["Tempo"].apply(lambda tempo: int(tempo.split(":")[0])*60 + int(tempo.split(":")[1]))

display(df_base)

,Interface,Tempo,Gênero,Tempo (seg)
0,A,3:00,Masculino,180
1,A,3:02,Masculino,182
2,A,2:58,Masculino,178
3,A,3:01,Feminino,181
4,A,3:03,Masculino,183
5,A,2:59,Feminino,179
6,A,3:04,Masculino,184
7,A,2:57,Masculino,177
8,A,3:00,Masculino,180
9,A,3:01,Masculino,181


### <center>Retirando Outlier

In [189]:
df_base = df_base[df_base['Tempo (seg)'] != 169].reset_index(drop=True)

print(df_base)

tempo_a = df_base[df_base["Interface"] == "A"]["Tempo (seg)"]
tempo_b = df_base[df_base["Interface"] == "B"]["Tempo (seg)"]

   Interface Tempo     Gênero  Tempo (seg)
0          A  3:00  Masculino          180
1          A  3:02  Masculino          182
2          A  2:58  Masculino          178
3          A  3:01   Feminino          181
4          A  3:03  Masculino          183
5          A  2:59   Feminino          179
6          A  3:04  Masculino          184
7          A  2:57  Masculino          177
8          A  3:00  Masculino          180
9          A  3:01  Masculino          181
10         B  2:59   Feminino          179
11         B  2:55   Feminino          175
12         B  2:59   Feminino          179
13         B  2:57  Masculino          177
14         B  2:58   Feminino          178
15         B  3:03  Masculino          183
16         B  2:56   Feminino          176
17         B  2:58  Masculino          178
18         B  2:56  Masculino          176


### <center>Test-t studend usando scipy.stats

In [190]:
# Realizar o teste t
t_stat, p_value = ttest_ind(tempo_a, tempo_b, equal_var=False, alternative="greater")

print(f"Estatística t: {t_stat:.4f}")
print(f"Valor p: {p_value:.9f}")

if p_value < 0.05:
    print("Há uma diferença estatisticamente significativa entre as interfaces A e B.")
else:
    print("Não há uma diferença estatisticamente significativa entre as interfaces A e B.")

Estatística t: 2.4945
Valor p: 0.011830431
Há uma diferença estatisticamente significativa entre as interfaces A e B.


### <center>Test-t studend usando pingouin

In [191]:
# Realizando o teste t com Pingouin
result = pg.ttest(tempo_a, tempo_b, paired=False, alternative='greater', correction=True)

# Exibindo os resultados
print(result[['T', 'p-val', 'dof']])

               T    p-val        dof
T-test  2.494501  0.01183  16.363695


### <center>Calculando "a mão" para visualizar melhor se realmente há uma diminuição significativa do tempo da tela de teste A para a tela de teste B

In [192]:
from math import sqrt, ceil, floor

media_a = tempo_a.mean()
media_b = tempo_b.mean()

var_a = tempo_a.var()
var_b = tempo_b.var()

n_a = len(tempo_a)
n_b = len(tempo_b)

t_test = (media_a - media_b) / sqrt((var_a / n_a) + (var_b / n_b))

baixo_df = (((var_a / n_a) ** 2) / (n_a - 1) + ((var_b / n_b) ** 2) / (n_b - 1))

df = ((var_a / n_a + var_b / n_b) ** 2) / baixo_df

print(f"Média A: {media_a}")
print(f"Média B: {media_b}\n")
print(f"Variancia A: {var_a}")
print(f"Variancia B: {var_b}\n")
print(f"T-teste: {t_test}\n")
print(f"Degrees of freedom (Grau de liberdade): {df}")
print(f"Df inteiro para cima: {ceil(df)}")
print(f"Df inteiro para baixo: {floor(df)}")

Média A: 180.5
Média B: 177.88888888888889

Variancia A: 4.722222222222222
Variancia B: 5.611111111111112

T-teste: 2.4945009944410965

Degrees of freedom (Grau de liberdade): 16.363695384016104
Df inteiro para cima: 17
Df inteiro para baixo: 16


### <center>Verificando se houve aumento significativo entre ambos os DF's


#### Batendo na tabela foram achados, para α = 0.05 e DF = 14 ou 15, os respectivos valores de t-critico, 1.761 e 1.753

In [195]:
if t_test > 1.761 and 1.753:
    print("O T-test calculado demonstra evidencias de que a interface B teve um tempo de execução de tarefas menor em comparação com a interface A com 95% de confincança!")
else:
    print("O T-test calculado esta abaixo dos valores criticos, ou seja, não é possível provar que de fato houve uma diminuição de A para B com 95% de confincança!")
    
if t_test > 2.624 and 2.602:
    print("O T-test calculado demonstra evidencias de que a interface B teve um tempo de execução de tarefas menor em comparação a interface A com 99% de confincança!")
else:
    print("O T-test calculado esta abaixo dos valores criticos, ou seja, não é possível provar que de fato houve uma diminuição de A para B com 99% de confincança!")

O T-test calculado demonstra evidencias de que a interface B teve um tempo de execução de tarefas menor em comparação com a interface A com 95% de confincança!
O T-test calculado esta abaixo dos valores criticos, ou seja, não é possível provar que de fato houve uma diminuição de A para B com 99% de confincança!


### <center>Base redistribuida

In [194]:
px.box(df_base, x="Interface", y="Tempo (seg)")

Essa é a nossa base redistribuída sem o outlier, e foi possível observar que, ao utilizarmos um nível de significância de 5%, podemos afirmar que realmente houve uma diminuição no desempenho do teste A em relação ao teste B.

Porém, ao adotarmos um nível de significância de 1%, essa afirmação não se mantém, pois o valor-p é maior que 0,01 (sendo 0,01183), enquanto a estatística do teste t (2,949501) é inferior ao valor crítico de 2,624 (com 14 graus de liberdade) e 2,602 (com 15 graus de liberdade). Assim, não temos evidências suficientes para afirmar, com 99% de confiança, que o desempenho do teste B foi realmente superior ao do teste A.

Ou seja, podemos concluir que a diferença estatistica entre os dois teste é significativo, mas não de um forma muito forte.

E é possível observar praticamente a mudança que um pequene outlier estava tendo no resultado final de nosso teste